In [ ]:
import matplotlib
# %matplotlib inline
# print("BACKEND: ", matplotlib.get_backend())
# if matplotlib.get_backend() != "macosx":
#   print("Changing backend to macosx")
#   matplotlib.use('macosx')


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# matplotlib.use("Agg")

from ase.visualize import view

from ase import Atoms
from ase.build import bulk
from ase.io import read, write
from agox.databases import Database
from agox.environments import Environment
from agox.utils.graph_sorting import Analysis

import numpy as np

In [ ]:
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rc('text', usetex=True)
plt.rc('font', family='cmr10', size=12)
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
from agox.models.descriptors import SOAP

local_descriptor = local_descriptor = SOAP.from_species(["C", "Mg", "O"], r_cut=6.0)


In [ ]:
import glob
from chgnet.model import CHGNetCalculator
calc = CHGNetCalculator()
# # for each POSCAR_[0-9] in iteration*/, read in to Atoms() object and attach calculator
# poscar_files = glob.glob("iteration*/POSCAR_[0-9]")
# atoms_list = []
# for poscar_file in poscar_files:
#   atoms = read(poscar_file)
#   atoms.calc =calc
#   atoms_list.append(atoms)


In [ ]:
seed = 0
identifier = ""
# min_energy = -9.064090728759766

In [ ]:
def compare_cells_and_carbon(structure, host):
    """Check if the structure and host have the same cell and same number of carbon atoms."""
    # Compare cell parameters
    cell_match = np.allclose(structure.cell, host.cell, atol=1e-5)

    # Count number of carbon atoms
    carbon_count_structure = sum(atom.symbol == 'C' for atom in structure)
    carbon_count_host = sum(atom.symbol == 'C' for atom in host)

    return cell_match and (carbon_count_structure == carbon_count_host)

In [ ]:
poscar_files = glob.glob("DC-MgO_hosts/POSCAR_*")
hosts = []
for poscar_file in poscar_files:
  host = read(poscar_file)
  host.calc =calc
  hosts.append(host)

In [ ]:
MgO = Atoms("Mg4O4", positions=[
        [0.0, 0.0, 0.0],
        [0.0, 2.097, 2.097],
        [2.097, 0.0, 2.097],
        [2.097, 2.097, 0.0],
        [0.0, 0.0, 2.097],
        [0.0, 2.097, 0.0],
        [2.097, 0.0, 0.0],
        [2.097, 2.097, 2.097],
    ], cell=[4.1940, 4.1940, 4.1940], pbc=True)
MgO.calc = calc

In [ ]:
unrlxd_structures = read("DTMP"+identifier+"/unrlxd_structures_seed"+str(seed)+".traj", index=":")
for structure in unrlxd_structures:
  structure.calc = calc

In [ ]:
rlxd_structures = read("DTMP"+identifier+"/rlxd_structures_seed"+str(seed)+".traj", index=":")
for structure in rlxd_structures:
  structure.calc = calc
# min_energy = min([structure.get_potential_energy()/len(structure) for structure in rlxd_structures])

In [ ]:
unrlxd_delta_en_per_atom = []
for structure in unrlxd_structures:
  host_energy = None
  area = None
  energy = structure.get_potential_energy()
  # compare cell of structure to cell of host and find matching host cell, i.e. same cell and same number of carbon atoms
  for host in hosts:
    if compare_cells_and_carbon(structure, host):
      host.calc = calc
      host_energy = host.get_potential_energy()
      area = np.linalg.norm(np.cross(host.cell[0], host.cell[1]))
      break
  if host_energy is None:
    print("No matching host for structure")
    continue
  energy -= host_energy
  energy -= MgO.get_potential_energy() * sum(atom.symbol == 'Mg' for atom in structure)
  energy /= 2 * area
  unrlxd_delta_en_per_atom.append(energy)
print("Unrelaxed min energy: ", min(unrlxd_delta_en_per_atom))
    

In [ ]:
rlxd_delta_en_per_atom = []
for structure in rlxd_structures:
  host_energy = None
  area = None
  energy = structure.get_potential_energy()
  # compare cell of structure to cell of host and find matching host cell, i.e. same cell and same number of carbon atoms
  for host in hosts:
    if compare_cells_and_carbon(structure, host):
      host.calc = calc
      host_energy = host.get_potential_energy()
      area = np.linalg.norm(np.cross(host.cell[0], host.cell[1]))
      break
  if host_energy is None:
    print("No matching host for structure")
    continue
  energy -= host_energy
  energy -= MgO.get_potential_energy() * sum(atom.symbol == 'Mg' for atom in structure)
  energy /= 2 * area
  rlxd_delta_en_per_atom.append(energy)
print("Relaxed min energy: ", min(rlxd_delta_en_per_atom))
    

In [ ]:
# # Calculate energies per atom for each unique structure
# energies_per_atom = [structure.get_potential_energy() / len(structure) for structure in unrlxd_structures]
# unrlxd_delta_en_per_atom = np.array(energies_per_atom) - min_energy
# print("Unrelaxed min energy: ", np.min(energies_per_atom))

In [ ]:
# energies_per_atom = [structure.get_potential_energy() / len(structure) for structure in rlxd_structures]
# rlxd_delta_en_per_atom = np.array(energies_per_atom) - min_energy
# print("Relaxed min energy: ", np.min(energies_per_atom))

In [ ]:
# if abs( np.min(energies_per_atom) - min_energy ) > 5e-2:
#   print("Min energy per atom: ", np.min(energies_per_atom), min_energy)
#   print("Minimum energy per atom is not zero. Check the energy calculation.")
#   # exit()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
unrlxd_super_atoms = []
for structure in unrlxd_structures:
  unrlxd_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
rlxd_super_atoms = []
for structure in rlxd_structures:
  rlxd_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
rlxd_string = "rlxd" # which dataset to fit the PCA model to

In [ ]:
# save pca model
import pickle
if True:
  pca.fit(np.squeeze([arr for arr in rlxd_super_atoms]))
  with open("pca_model"+identifier+"_all_rlxd_"+str(seed)+".pkl", "wb") as f:
    pickle.dump(pca, f)

# load pca model
with open("pca_model"+identifier+"_all_"+rlxd_string+"_0.pkl", "rb") as f:
  pca = pickle.load(f)

In [ ]:
unrlxd_X_reduced = pca.transform(np.squeeze([arr for arr in unrlxd_super_atoms]))
rlxd_X_reduced = pca.transform(np.squeeze([arr for arr in rlxd_super_atoms]))

In [ ]:
min_energy_index = np.argmin(rlxd_delta_en_per_atom)
print(min_energy_index)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))

plt.subplots_adjust(wspace=0.05, hspace=0)

max_en = min(3.5, max(np.max(unrlxd_delta_en_per_atom), np.max(rlxd_delta_en_per_atom)))

axes[0].scatter(unrlxd_X_reduced[:, 0], unrlxd_X_reduced[:, 1], c=unrlxd_delta_en_per_atom, cmap="viridis", vmin = 0, vmax = max_en)
axes[1].scatter(rlxd_X_reduced[:, 0], rlxd_X_reduced[:, 1], c=rlxd_delta_en_per_atom, cmap="viridis", vmin = 0, vmax = max_en)
for ax in axes:
  # ax.scatter(rlxd_X_reduced[471, 0], rlxd_X_reduced[471, 1], s=200, edgecolor=[1.0, 0.5, 0.5, 0.8], facecolor='none', linewidth=2)
  ax.scatter(rlxd_X_reduced[min_energy_index, 0], rlxd_X_reduced[min_energy_index, 1], s=200, edgecolor='red', facecolor='none', linewidth=2)

fig.text(0.5, 0.04, 'Principal Component 1', ha='center', fontsize=15)
axes[0].set_ylabel('Principal Component 2', fontsize=15)
axes[0].set_title('Unrelaxed')
axes[1].set_title('Relaxed')
if identifier == "_VASP":
  if rlxd_string == "rlxd":
    xlims = [-11, 8]
    ylims = [-5, 6]
  else:
    xlims = [-9, 13]
    ylims = [-7, 12]
else:
  if rlxd_string == "rlxd":
    xlims = [-250, 150]
    ylims = [-10, 70]
  else:
    xlims = [-600, 600]
    ylims = [-100, 100]
  # if rlxd_string == "rlxd":
  #   xlims = [-25, 10]
  #   ylims = [-12, 6]
  # else:
  #   xlims = [-5, 13]
  #   ylims = [-6.5, 13]

for ax in axes:
  ax.tick_params(axis='both', direction='in')
  ax.set_xlim(xlims)
  ax.set_ylim(ylims)

# unify tick labels
xticks = axes[0].get_xticks()
xticks = xticks[(xticks >= xlims[0]) & (xticks <= xlims[1])]

axes[0].set_xticks(xticks)
axes[1].set_xticks(xticks)
axes[1].set_yticklabels([])
axes[0].tick_params(axis='x', labelbottom=True, top=True)
axes[1].tick_params(axis='x', labelbottom=True, top=True)
axes[0].tick_params(axis='y', labelbottom=True, right=True)
axes[1].tick_params(axis='y', labelbottom=True, right=True)

# Make axes[0] and axes[1] the same width
axes[0].set_box_aspect(1.7)
axes[1].set_box_aspect(1.7)

# Add colorbar next to the axes
cbar = fig.colorbar(axes[1].collections[0], ax=axes, orientation='vertical', fraction=0.085, pad=0.02)
cbar.set_label('Formation energy (eV/atom)', fontsize=15)

plt.savefig('C-MgO_RAFFLE'+identifier+'_pca_'+rlxd_string+'_fit_seed'+str(seed)+'.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')

# interface formation energy, i.e. (system - graphite host - bulk MgO)/2A